<a href="https://colab.research.google.com/github/francescopatane96/RNA-seq_Pipeline/blob/main/clusterprofiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("clusterProfiler")

In [ ]:
library(clusterProfiler)


In [ ]:
organism = "org.Dm.eg.db"
BiocManager::install(organism, character.only = TRUE)
library(organism, character.only = TRUE)

In [ ]:
genelistup <- read.csv('your_file.csv', header=F, sep=' ')
genelist_UP <- genelistup$V2                                    #select gene column


genelist <- bitr(genelist_UP, fromType = "SYMBOL",          
                toType=c("ENTREZID", "ENSEMBL"),
                OrgDb="org.Hs.eg.db")

gene_list=genelist$ENSEMBL

gene_list_UP=as.character(gene_list)

GO classification.

- Functional Profile of a gene set at specific GO level. Given a vector of genes, this function will return the GO profile at a specific level

In [ ]:
ggo <- groupGO(gene=gene_list_UP, 
               OrgDb="org.Hs.eg.db",
               keyType='ENSEMBL',
               ont="BP", 
               level=3,
               readable=T)
head(ggo)

In [ ]:
barplot(ggo, drop=TRUE, showCategory=12)

GO ORA (over-representation analysis)

In [ ]:
ego <- enrichGO(gene          = gene_list_UP,
                OrgDb         = org.Hs.eg.db,
                keyType       = "ENSEMBL",
                ont           = "BP",
                pAdjustMethod = "BH",
                pvalueCutoff  = 0.1,
                qvalueCutoff  = 0.1,
                readable      = TRUE)
head(ego)

In [ ]:
library(DOSE)
library(enrichplot)
library(wordcloud)

In [ ]:
upsetplot(ego)

In [ ]:
wcdf<-read.table(text=ego$GeneRatio, sep = "/")[1]
wcdf$term<-ego[,2]
wordcloud(words = wcdf$term, freq = wcdf$V1, scale=(c(4, .1)), colors=brewer.pal(8, "Dark2"), max.words = 25)

In [ ]:
barplot(ego, 
        drop = TRUE, 
        showCategory = 10, 
        title = "GO Biological Pathways",
        font.size = 8)

In [ ]:
dotplot(ego)

In [ ]:
emapplot(ego)

In [ ]:
goplot(ego, showCategory = 10)

In [ ]:
plotGOgraph(ego)


In [ ]:
# categorySize can be either 'pvalue' or 'geneNum'
cnetplot(ego, categorySize="pvalue", foldChange=gene_list)

GO gene set enrichment analysis

- geneList input would be a ranked gene list, with 3 features: numeric vector,
named vector and sorted vector.
if import data from csv, you must have 2 columns,
one for gene ID and another one for fold change

In [ ]:
d <- read.csv('E:/611911_fin/counts/DEG_UP_padj_min01_611911.csv', header=T, sep=',')

In [ ]:
## assume 2nd column is ID
## 3rd column is FC

## feature 1: numeric vector
geneList = d[,2]

## feature 2: named vector
names(geneList) = as.character(d[,1])

## feature 3: decreasing order
geneList = sort(geneList, decreasing = TRUE)

In [ ]:
gse <- gseGO(geneList=geneList, 
             OrgDb='org.Hs.eg.db',
             ont ="ALL", 
             keyType = "ENSEMBL", 
             nPerm = 10000, 
             minGSSize = 3, 
             maxGSSize = 800, 
             pvalueCutoff = 0.1, 
             verbose = TRUE, 
             pAdjustMethod = "none")

head(gse)

In [ ]:
#Dotplot

dotplot(gse, showCategory=15, split=".sign") 

In [ ]:
#The goplot() function can accept the output of enrichGO and visualize the enriched 
#GO induced graph.
goplot(ego)

In [ ]:
# Use the `Gene Set` param for the index in the title, and as the value for geneSetId
gseaplot(gse, by = "all", title = gse$Description[1], geneSetID = 1)

KEGG ORA analysis

In [ ]:
search_kegg_organism('sap', by='kegg_code')
search_kegg_organism('sapiens', by='scientific_name')
#hsa

#Input ID type can be kegg, ncbi-geneid, ncbi-proteinid or uniprot

prova <- bitr(gene_list, fromType = "ENSEMBL",
                 toType=c("UNIPROT"),
                 OrgDb="org.Hs.eg.db")

In [ ]:
kegg_type <- bitr_kegg(prova$UNIPROT, fromType='uniprot', toType='kegg', organism='hsa')

kk <- enrichKEGG(gene         = kegg_type$kegg,
                 organism     = 'hsa',
                 pvalueCutoff = 0.05)
head(kk)


KEGG GSEA

In [ ]:
#geneList must be a decreasing sorted vector
kk2 <- gseKEGG(geneList     = kegg_type$kegg,
               organism     = 'hsa',
               nPerm        = 1000,
               minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
head(kk2)

In [ ]:
gseaplot(kk2, geneSetID = "hsa04145")

KEGG module ORA

In [ ]:
mkk <- enrichMKEGG(gene = kegg_type$kegg,
                   organism = 'hsa',
                   pvalueCutoff = 1,
                   qvalueCutoff = 1)
head(mkk)           

KEGG module GSEA

In [ ]:
mkk2 <- gseMKEGG(geneList = geneList,
                 organism = 'hsa',
                 pvalueCutoff = 1)
head(mkk2)

visualize enriched KEGG pathway

In [ ]:
browseKEGG(kk, 'hsa05205')

In [ ]:
library("pathview")
 pathview(gene.data=kegg_type$kegg,
                     pathway.id = "hsa05205",
                     species    = "hsa",
                     limit      = list(gene=max(abs(geneList)), cpd=1))

In [ ]:
# Produce the native KEGG plot (PNG)
 dme <- pathview(gene.data=kegg_type$kegg, pathway.id="hsa05205", species = 'hsa')
 

In [ ]:
# Produce a different plot (PDF) 
 dme <- pathview(gene.data=kegg_type$kegg, pathway.id="hsa05205", species = 'hsa', kegg.native = F)

In [ ]:
barplot(kk, 
         showCategory = 10, 
         title = "Enriched Pathways",
         font.size = 8)

In [ ]:
dotplot(kk, 
         showCategory = 10, 
         title = "Enriched Pathways",
         font.size = 8)

In [ ]:
 # categorySize can be either 'pvalue' or 'geneNum'
 cnetplot(kk, categorySize="pvalue", foldChange=gene_list)

WikiPathways analysis

In [ ]:
#ORA
 enrichWP(genelist$ENTREZID, organism = "Homo sapiens") 

In [ ]:
#GSEA (decreasing sorted vector)
 gseWP(gene_list, organism = "Homo sapiens")

REACTOME

In [ ]:
library(ReactomePA)

In [ ]:
#ORA
 x <- enrichPathway(gene=genelist$ENTREZID, pvalueCutoff = 0.05, readable=TRUE)
 head(x)

In [ ]:
#GSEA
 y <- gsePathway(geneList, 
                 pvalueCutoff = 0.2,
                 pAdjustMethod = "BH", 
                 verbose = FALSE)
 head(y)

In [ ]:
#pathway visualization
 
 
 viewPathway("E2F mediated regulation of DNA replication", 
             readable = TRUE, 
             foldChange = d$V2)

disease enrichment analysis (DO)

In [ ]:
library(DOSE)

In [ ]:
#ORA
 x <- enrichDO(gene          = genelist$ENTREZID,
               ont           = "DO",
               pvalueCutoff  = 0.05,
               pAdjustMethod = "BH",
               universe      = names(genelist$ENTREZID),
               minGSSize     = 5,
               maxGSSize     = 500,
               qvalueCutoff  = 0.05,
               readable      = FALSE)
 head(x)

In [ ]:
#GSEA
 gene2 <- names(genelist$ENTREZID)
 ncg <- enrichNCG(gene2) 
 head(ncg)